In [1]:
import os
%pwd

'/Users/harendrakumar/Documents/Demand_forecast/research'

In [2]:
os.chdir("../")
%pwd

'/Users/harendrakumar/Documents/Demand_forecast'

In [3]:
os.environ['MLFLOW_TRACKING_URI']= "https://dagshub.com/developerdatascience/Demand_forecast.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="developerdatascience"
os.environ['MLFLOW_TRACKING_PASSWORD']="3f1565218d6ec2d42985b700ffecee09de158b93"

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    train_data_path: Path
    model_path: Path
    fcst_params: dict
    all_inventory_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [6]:
from Demand_Forecast.constants import *
from Demand_Forecast.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath= CONFIG_FILE_PATH,
                 schema_filepath= SCHEMA_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifact_roots])
    
    def get_model_evaluation_config(self) ->ModelEvaluationConfig:
        config = self.config.model_evaluation
        fcst_params = self.params.Forecast
        inventory_params = self.params.Inventory
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        return ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            train_data_path= config.train_data_path,
            model_path= config.model_path,
            fcst_params= fcst_params,
            all_inventory_params=inventory_params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mlflow_uri= "https://dagshub.com/developerdatascience/Demand_forecast.mlflow"
        )

In [8]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib


[2023-10-21 16:54:11,106: INFO: utils: NumExpr defaulting to 8 threads.]


In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config
    
    @staticmethod
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    @staticmethod
    def load_data(df):
        df = df[['Date', 'Product_ID', 'Demand', 'Inventory']]
        df = df.set_index('Date')
        df = df['Demand']
        return df
    
    def log_into_mlflow(self):
        test = self.load_data(df=pd.read_csv(self.config.test_data_path))
        train = self.load_data(df=pd.read_csv(self.config.train_data_path))
        model = joblib.load(self.config.model_path)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            model_fit = model.fit(disp=False)
            predictions = model_fit.predict(start=len(train), end=len(train) + len(test) -1)
            predictions = predictions.astype(int)

            (rmse, mae, r2) = self.eval_metrics(actual=test, pred=predictions)

            #saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path = Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.fcst_params)
            mlflow.log_params(self.config.all_inventory_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="Time Series-SARIMAX")
            else:
                mlflow.sklearn.log_model(model, "model")

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-10-21 16:54:12,391: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-21 16:54:12,394: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-21 16:54:12,396: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-21 16:54:12,397: INFO: common: created directory at: artifacts]
[2023-10-21 16:54:12,398: INFO: common: created directory at: artifacts/model_evaluation]
[2023-10-21 16:54:22,118: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


/Users/harendrakumar/.local/share/virtualenvs/ETL-6mpoCg48/lib/python3.10/site-packages/_distutils_hack/__init__.py:30: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'Time Series-SARIMAX' already exists. Creating a new version of this model...
2023/10/21 16:54:57 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: Time Series-SARIMAX, version 2
Created version '2' of model 'Time Series-SARIMAX'.
